# Setup

From: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content. Conversations can be as short as one message or many back and forth turns.

How to set openai API key if using conda environments:
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#setting-environment-variables

1. `conda activate [name of env]`
2. `conda env config vars set OPENAI_API_KEY=[openai api key]`
3. `conda activate [name of env]`
4. restart vscode/kernel

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from openai import OpenAI
client = OpenAI()

def llm(prompt, stop=["\n"]):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=100,
        # top_p=1,
        # sample=False,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=stop,
        seed=12345
    )
    return completion.choices[0].message.content

In [4]:
llm("What is the capital of France? and write me a poem")

'The capital of France is Paris,'

In [5]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [6]:
print(env.print_sections('1920'))

*: Events
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Births
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Deaths
**: January
**: February
**: March
**: April
**: May
**: June
**: July
**: August
**: September
**: October
**: November
**: December
**: Date unknown
*: Nobel Prizes
*: References
**: Primary sources and year books



In [7]:
# additional examples
# read text file
with open("bonn.txt", "r") as file:
    additional_examples = ''.join( file.readlines())
    # examples = [x.strip() for x in examples]

In [8]:
print(additional_examples)

Question: What is the population of the city where the Akademisches Kunstmuseum is?
Thought 1: It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.
Action 1: Search[Akademisches Kunstmuseum]
Observation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bonn with more than 2,700 plaster casts of antique statues and reliefs, and over 25,000 originals. It is located in a neoclassical building at the southern end of the Hofgarten, near the Electoral Palace. During the renovation of the historic building, the museum can currently be visited at Römerstraße 164 in 53117 Bonn.. The museum was founded in 1818 and has one of the largest collections of plaster casts of ancient Greek and Roman sculptures in the world.
Thought 2: 

## Interactive demo

In [65]:
# test environment manually
question_idx = 0
s = ""
question = env.reset(idx=question_idx)
s += question + "\n"
print(question)

for i in range(10):
    thought_str = f'Thought {i+1}:'
    thought = input(thought_str)
    print(thought_str, thought)
    s += thought_str + " " + thought + "\n"

    action_str = f"Action {i+1}:"
    action = input(action_str)
    print(action_str, action)
    s += action_str + " " + action + "\n"
    
    obs, r, done, info = step(env, action[0].lower() + action[1:])
    print(f"Observation {i+1}:", obs)
    s += f"Observation {i+1}: {obs}\n"

    if done:
        break
if not done:
    obs, r, done, info = step(env, "finish[]")
    print("Observation:", obs)
# save s to file using jsonl format
import json
with open(f"hotpotqa/{question_idx}.jsonl", "w") as file:
    file.write(json.dumps(s))
    file.write("\n")

Question: Were Scott Derrickson and Ed Wood of the same nationality?
Thought 1: test
Action 1: sections_by_title[ed wood]
Observation 1: Invalid action: sections_by_title[ed wood]
Thought 2: 
Action 2: 


IndexError: string index out of range

In [53]:
info

{'steps': 4, 'answer': None}

# ReAct

In [12]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
# (4) PrintSections[entity], which returns the sections of the Wikipedia page for the entity.

webthink_prompt = instruction + webthink_examples # + additional_examples + '\n'


In [13]:
print(webthink_examples)


Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions:
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[

In [14]:
print(webthink_prompt)

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern se

In [15]:

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
            
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [16]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[:100]:
# i = 6554
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Question: What movie did actress Irene Jacob complete before the American action crime thriller film directed by Stuart Bird?
Thought 1: I need to search for information about actress Irene Jacob and the American action crime thriller film directed by Stuart Bird to find out the movie Irene Jacob completed before that. Let's start by searching for Irene Jacob.
Action 1: Search[Irene Jacob]
Observation 1: Irène Marie Jacob (born 15 July 1966) is a French-Swiss actress known for her work with Polish film director Krzysztof Kieślowski.[2] She won the 1991 Cannes Film Festival Award for Best Actress for the Kieślowski film The Double Life of Veronique, and was nominated for the BAFTA Award for Best Actress in a Leading Role for her 1994 film Three Colours: Red. Her other film appearances include The Secret Garden (1993), Beyond the Clouds (1995), U.S. Marshals (1998), and Eternity (2016).. Irène Jacob was born in Suresnes, Hauts-de-Seine, a western suburb of Paris.[3] The youngest chi

In [17]:
# dump the results using pretty json
import json
with open('results/hotpotqa-gpt-3.5-turbo-examples100.json', 'w') as f:
    json.dump(infos, f, indent=4)